In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from adan_pytorch import Adan
from madgrad import MADGRAD
from utils import DEVICE, LOSS_CRITERIA
import time
from transformers import HubertConfig, HubertForSequenceClassification
from utils import ONLY_10_LABELS
import matplotlib.pyplot as plt
from utils import DEVICE, AudioDataset
from torch.utils.data import DataLoader
import optuna
import pandas as pd
import schedulefree
torch.cuda.empty_cache()

/home/mlspeech/royweber/anaconda3/envs/roy_lang/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 20
torch.manual_seed(seed)

In [3]:
batch_size = 64
num_labels = 10 if ONLY_10_LABELS else 50
epochs = 10
device=DEVICE
loss_criteria = LOSS_CRITERIA
NTRAILS = 150

train_dataset = AudioDataset(kind='train')
val_dataset = AudioDataset(kind='val')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

dataloaders = {
    'train':train_loader,
    'val':val_loader
}



def optune_optimizer_for_model(trial, optimizer_name):
    
    config = HubertConfig(
    hidden_size=128,
    num_hidden_layers=2,
    num_attention_heads=4,
    intermediate_size=256,
    num_labels=num_labels,
    conv_dim=(512, 512, 512),
    conv_stride=(5, 2, 2),
    conv_kernel=(10, 3, 3),
    )

    hubert_model = HubertForSequenceClassification(config)
    
    model = hubert_model.to(DEVICE)
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)  # log=True, will use log scale to interplolate between lr
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name == 'SGD':
        sgd_momentum = trial.suggest_float("sgd_momentum", 1e-1, 1)
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=sgd_momentum, nesterov=True)
    elif optimizer_name == 'Adan':
        beta1 = trial.suggest_float("beta1", 1e-3, 1e-1)
        beta2 = trial.suggest_float("beta2", 1e-3, 1e-1)
        beta3 = trial.suggest_float("beta3", 1e-3, 1e-1)
        optimizer = Adan(model.parameters(),lr = lr,
            betas = (beta1, beta2, beta3), 
            weight_decay = 0.02         # weight decay 0.02 is optimal per author
        )
    elif optimizer_name == 'Madgrad':
        lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
        madgrad_momentum = trial.suggest_float("madgrad_momentum", 8e-1, 1)
        optimizer = MADGRAD(params=model.parameters(), lr=lr, momentum=madgrad_momentum, weight_decay=0, eps= 1e-06, decouple_decay=False)
    
    elif optimizer_name == 'Schedulefree':
        lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
        optimizer = schedulefree.AdamWScheduleFree(model.parameters(), lr=1e-3)
    

    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        print('-' * 10)
        
        model.train()
        
        # Iterate over data.
        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.set_grad_enabled(True):
                # Get model outputs and calculate loss
                logits_h = model(inputs).logits.type(torch.cuda.FloatTensor)
                labels = labels.type(torch.cuda.FloatTensor)
                loss = loss_criteria(logits_h.view(-1), labels.view(-1))
                # loss = loss_criteria(outputs, labels)

                _, preds = torch.max(logits_h, 1)
                _, real_labels = torch.max(labels, 1)

                # backward + optimize only if in training phase
                # zero the parameter gradients
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        
        model.eval()
        running_corrects = 0.0
        for inputs, labels in dataloaders['val']:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(False):
                    # Get model outputs and calculate loss
                    
                    logits_h = model(inputs).logits.type(torch.cuda.FloatTensor)
                    labels = labels.type(torch.cuda.FloatTensor)
                    loss = loss_criteria(logits_h.view(-1), labels.view(-1))

                    _, preds = torch.max(logits_h, 1)
                    _, real_labels = torch.max(labels, 1)
                    

                # statistics
                running_corrects += torch.sum(preds == real_labels.data)

        epoch_acc = running_corrects.double() / len(dataloaders['val'].dataset)

        # report back to Optuna how far it is (epoch-wise) into the trial and how well it is doing (accuracy)
        trial.report(epoch_acc, epoch)  

        # then, Optuna can decide if the trial should be pruned
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return epoch_acc

In [4]:
# now we can run the experiment
sampler = optuna.samplers.TPESampler()
configurations = [
                {'optimizer_name':'Adam',
                   'study_name':'adam_study'},
                  {'optimizer_name':'SGD',
                   'study_name':'sgd_study'},
                  {'optimizer_name':'Adan',
                   'study_name':'adan_study'},
                  {'optimizer_name':'Madgrad',
                   'study_name':'madgrad_study'},
                  {'optimizer_name':'Schedulefree',
                   'study_name':'scheduleFree_study'},
                  ]
studies = {}
for config in configurations:
    
    study = optuna.create_study(study_name=config['study_name'], direction="maximize", sampler=sampler)
    study.optimize(lambda trial: optune_optimizer_for_model(trial, config['optimizer_name']), n_trials=NTRAILS, timeout=16*60) #trial.report
    
    study_csv = f'{config["study_name"]}.csv'
    study.trials_dataframe().to_csv(study_csv)
    studies[config["study_name"]] = study_csv
    
    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))
    print("Best trial:")

    trial = study.best_trial

    print("  Value: ", trial.value)
    print("  Params: ")

    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-08-12 08:41:18,503] A new study created in memory with name: adam_study


Epoch 0/9
----------
Epoch 1/9
----------
Epoch 2/9
----------
Epoch 3/9
----------
Epoch 4/9
----------
Epoch 5/9
----------
Epoch 6/9
----------
Epoch 7/9
----------
Epoch 8/9
----------
Epoch 9/9
----------


[I 2024-08-12 08:41:41,523] Trial 0 finished with value: 0.4625 and parameters: {'lr': 6.385644044670937e-05}. Best is trial 0 with value: 0.4625.


Epoch 0/9
----------
Epoch 1/9
----------
Epoch 2/9
----------
Epoch 3/9
----------
Epoch 4/9
----------
Epoch 5/9
----------
Epoch 6/9
----------
Epoch 7/9
----------
Epoch 8/9
----------
Epoch 9/9
----------


[I 2024-08-12 08:42:02,621] Trial 1 finished with value: 0.1 and parameters: {'lr': 0.01971188337614859}. Best is trial 0 with value: 0.4625.


...

Epoch 0/9
----------


[I 2024-08-12 09:44:57,341] Trial 148 pruned. 


Epoch 0/9
----------


[I 2024-08-12 09:44:59,666] Trial 149 pruned. 


Study statistics: 
  Number of finished trials:  150
  Number of pruned trials:  135
  Number of complete trials:  15
Best trial:
  Value:  0.6625000000000001
  Params: 
    lr: 8.277158497887823e-05
    madgrad_momentum: 0.9087751912981861


### After running optuna - recover results from studies files, it is better to take results from the logs up

In [7]:

def recover_study(file_name='study_trials.csv', distributions={}):
    trials_df = pd.read_csv(file_name)
    study = optuna.create_study()

    # Add trials back to the study
    for _, row in trials_df.iterrows():
        trial_params = {col.replace('params_',''): row[col] for col in trials_df.columns if col.startswith('params_')}
        trial = optuna.trial.create_trial(
            params=trial_params,
            distributions=distributions,
            value=row['value'],
            state=optuna.trial.TrialState.COMPLETE
        )
        # lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
        study.add_trial(trial)
    
    trial = study.best_trial

    print("  Value: ", trial.value)
    print("  Params: ")

    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    return study



In [1]:
for study_name, study_file in studies.items():
    if  study_name == 'adan_study':
        study1 = recover_study(study_file, distributions={
            'lr': optuna.distributions.FloatDistribution(1e-5, 1e-1),
            'beta1': optuna.distributions.FloatDistribution(1e-3, 1e-1),
            'beta2': optuna.distributions.FloatDistribution(1e-3, 1e-1),
            'beta3': optuna.distributions.FloatDistribution(1e-3, 1e-1)
            })
        # optuna.visualization.plot_param_importances(study)
    elif study_name == 'madgrad_study':
        study2 = recover_study(study_file, distributions={
            'lr': optuna.distributions.FloatDistribution(1e-5, 1e-1),
            'madgrad_momentum': optuna.distributions.FloatDistribution(8e-1, 1)
            })
    elif study_name == 'adam_study':
        study3 = recover_study(study_file, distributions={
            'lr': optuna.distributions.FloatDistribution(1e-5, 1e-1),
            })   
    elif study_name == 'sgd_study':
        study4 = recover_study(study_file, distributions={
            'lr': optuna.distributions.FloatDistribution(1e-5, 1e-1),
            'sgd_momentum':optuna.distributions.FloatDistribution(1e-1, 1),
            })  
    elif study_name == 'scheduleFree_study':
        study4 = recover_study(study_file, distributions={
            'lr': optuna.distributions.FloatDistribution(1e-5, 1e-1),
            })   
         
        
    else:
        continue